# <center> Econ 373: Computational Economics (with Python) </center>
## <center> Homework 11 (Individual) </center>

# <font color='red'>Name:</font>

# <font color='red'>Instructions:</font>
- Save all of your code to a .ipynb file (jupyter notebook file) and name it as **username_hw11.ipynb**. 
    - **You should remove any test cells/code that is outside of functions.**
    - Submit only username_hw7.ipynb file
- For each question, your file should contain a function labeled **q#** with input/output requirements specified below. 
    - The input refers to the arguments passed to the function. 
    - The output refers to what is returned by the function.
    - We may require output to file or screen within a function, but if that is the case it will be clearly specified.
    - Your functions may call other functions or classes that you create, but they have to be included in the file (i.e., the file that you submit should be self-contained).
    - If your function calls on functions from other libraries, you need to load them within the function (e.g., if you use the os library you should assume that it has been installed on the computer but it has not been imported before calling your function).
    

## Grading

- We will run your file by clicking Kernel--> Restart and Run All. You file should be able to reproduce all the results stored in your jupyter file. 

- We may also run your code by specifying q#(arg) in an empty cell. It should reproduce your stored results. 

Each question is graded on a 3-point scale + 1 point for following the instructions 
- 0 -- no or minimal work submitted (e.g., minor modification of the 'starting point')
- 1 -- some work done but there are errors running/executing the code or results are mostly incomplete
- 2 -- code runs, but results are either somewhat incomplete, incorrect, or there is clear room for improvement (e.g., no comments in the code, graphs are not labelled, etc.) 
- 3 -- all results complete and correct with clear commented code 

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as so
%matplotlib inline

In [15]:
import os
# To find your working directory:
%pwd 
# Code in case you want to change your working directory:  %cd
# for example: %cd "C:\Users\\Purdue\ComputationalEconomicsECON320\Week3_4\Group\"


'C:\\Users\\hp\\Downloads'

Consider the following variation of the finite horizon inter-temporal consumptions and saving problem discussed in the lecture.

You own a share of stock that pays a dividend $D_t\in\{1,5\}$ each period. Specifically, the dividend realization depends on how well the company does: when the company does well you get $5$ and when the company does poorly you get $1$. Assume that how well the company does is driven by the state of the economy and that the economic transition is modeled by the  Markov Chain process:

$$S \in \{Expansion,Recession\}$$

$$M=\left(\begin{array}{cc} 
.8 & .2\\ 
.4 & .6
\end{array}\right)$$

That is, when the economy is in the Expansion, the dividend is 5, but when the economy is in the recession, the dividend is 1.

Suppose that you begin with $x$ in your savings. Further suppose your utility of consuming $c$ units in a period is given by $u(c)=\sqrt{c}$, and that you live for $T=10$ periods.

In addition, suppose that the assets held at the end of period $t$ yield an interest (with interest rate $r=.05$) that is paid at the beginning.

Find the optimal policy.

# Question 1

Let's start from the last period as the lecture and define several functions that are relevant for this particular problem. 

Step 1: What should you do in period 10? -- spend everything

In this step, define a function, max_asset, to calculate the maximumal asset/wealth possible at period T:

- input: the starting asset (a1), the maximal income in each period (I), interest rate (r), the number of periods (T)
- output: a number 

*Hint: max_asset(0,5,0.05,10) = 62.889462677744156


Step 2: Suppose you are in period 9 and you have A=10 in total,and you are in expension, what you are maximizing is: 
$$ u(x)+ .8(Value10((A-x)*1.05+1)+.2(Value10((A-x)*1.05+5)$$

Now, suppose you are in period 9 and you have A=10 in total,and you are in recession, what are you maximizing? 

Write a function named negExpectedPayoff9 to calculate your utility in either expansion or recession. 
    
- input: your consumption (x), your asset (A), your state( S, 0 for Expansion and 1 for Recession)
- output: the corresponding utility 

*Hint: negExpectedPayoff9(x,A,0) = u(x)+ .8(Value10((A-x)*1.05+1)+.2(Value10((A-x)*1.05+5)

<br>

- Output: write out two functions in separate blocks as provided: 
    1. max_asset() 
    2. negExpectedPayoff


In [16]:
def max_asset(a1,I,r,T):
    
    asset = a1
    for _ in range(T):
        asset = asset * (1 + r) + I
    return asset

# Example calculation given in the question
max_asset_example = max_asset(0, 5, 0.05, 10)

print('Asset value now is',max_asset(0, 5, 0.05, 10))
    
    

Asset value now is 62.889462677744156


In [17]:
def negExpectedPayoff(x, A, S, r=0.05, I=5):

    # Utility of consuming x units in a given period
    utility = (x)**0.5
    
    # Value if in expansion
    if S == 0:
        # Next period's asset if in expansion
        next_asset_expansion = (A - x) * (1 + r) + I
        # Expected utility if in expansion
        utility += 0.8 * (next_asset_expansion * 0.5)
    else:
        # Next period's asset if in recession
        next_asset_recession = (A - x) * (1 + r) + I
        # Expected utility if in recession
        utility += 0.2 * (next_asset_recession * 0.5)
    
    return -utility  # Negating the utility as we are looking for the optimal policy

# Test the functions with an example
negExpectedPayoff_example = negExpectedPayoff(1, 10, 0, 0.05, 5)

max_asset_example, negExpectedPayoff_example

(62.889462677744156, -6.780000000000001)

# Question 2

Once you have defined the above two functions, let's start from finding the optimal policy in the previous period, and then continue the backward process to find the optimal policies and the value functions for each period. 


Step 3: Find the optimal policy in period 9 (i.e., for all possible A)

*Hint: your policy and value functions should reflect the two states.

Step 4: Find the optimal policy for every period t (continue working backwards)

Expected Output:

1. For step 3, use a separate block to output your period 9 policty and value functions

2. For step 4, follow the procedure discussed in the lecture and wrap up the whole backward deduction into a function called dynamic_program()

In [19]:
def Value(A):
    return A ** 0.5

In [20]:
# Step 3 Output goes here: 

NPOINTS = 15

policy9=np.zeros(shape=(NPOINTS,2),dtype=float)#The third entry corresponds to states of the economy
value9=np.zeros(shape=(NPOINTS,2),dtype=float)

for i in range(NPOINTS):
    asset_value = i 
    policy9[i, :] = asset_value  # Consume all assets
    value9[i, 0] = Value(asset_value)  # Value if in expansion
    value9[i, 1] = Value(asset_value)  # Value if in recession




In [24]:
# Step 4 Follow the procedure discussed in the lecture and wrap up the whole backward deduction into a function called dynamic_program
def dynamic_program(a1,I,r,T):
    NPOINTS = 15 
    assets = np.zeros(shape=(T,NPOINTS)) 
    policies = np.zeros(shape=(T,NPOINTS,2)) 
    values = np.zeros(shape=(T,NPOINTS,2))
    
    policies[T-1] = policy9
    values[T-1] = value9

    for t in range(T-2, -1, -1):
        if t == T:
            for i in range(NPOINTS):
                assets[t-1, i] = max_asset(a1, I, r, 1)  
                for S in [0, 1]:  # 0 for Expansion, 1 for Recession
                    values[t-1, i, S] = Value(assets[t-1, i]) 
                    policies[t-1, i, S] = assets[t-1, i]  # Consume all assets
        else:
            for i in range(NPOINTS):
                A = assets[t, i]  # Asset at the beginning of period t
                for S in [0, 1]:
                    # Assuming a simple grid search for the optimal consumption
                    x_vals = np.linspace(0, A, 100)
                    utilities = np.array([negExpectedPayoff(x, A, S, r, I) for x in x_vals])
                    best_x_idx = np.argmin(utilities)
                    policies[t-1, i, S] = x_vals[best_x_idx]  # Optimal policy
                    values[t-1, i, S] = -utilities[best_x_idx] 
                    # Update assets for the next period
                    assets[t-1, i] = A - policies[t-1, i, S]
        
    return assets,policies,values

In [25]:
# I will run the following function 

T=10 
a1=0 #Starting assets
I=5 #Income
r=0.05

assets, policies, values = dynamic_program(a1, I, r, T)

# Output the arrays for the last period
print("Policies for period 9:", policies[9])
print("Values for period 9:", values[9])

dynamic_program(a1,I,r,T)

Policies for period 9: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Values for period 9: [[2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]
 [2.  0.5]]


(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[